<div class="alert alert-block alert-success">
    <b><h2>Model Validation</h2></b> 
    <br> Bu işlemde veri seti eğitim ve test olmak üzere parçalara ayrılır. Burada ki amaç eğitilen modeli test verileri ile modelin başarımını ölçmektir.  <br>
        <img src="https://docs.splunk.com/images/thumb/3/3b/TrainTest.png/550px-TrainTest.png"  width=500 title="Karar Ağacı Örnek Yapısı" />
</div>

#### Bu bölümde işlenecek konular

1. [Veri seti oluşturma](#bir)
2. [Veri setini train test olarak parçalama](#iki)
3. [Karar ağacı modelini oluşturma ve eğitme](#uc)
4. [Karar ağacı test etme](#dort)
5. [Model kalitesi ölçme (MAE)](#bes)


In [47]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [48]:
data=pd.read_csv('./Data/melbourne-housing-snapshot/melb_data.csv')

In [49]:
data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [50]:
data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

- Veri içerisinde NaN içeren satırları siliyoruz

In [51]:
data=data.dropna(axis=0)

- Veri setindeki boş değerleri kontrol ediyoruz

In [52]:
data.isnull().sum()

Suburb           0
Address          0
Rooms            0
Type             0
Price            0
Method           0
SellerG          0
Date             0
Distance         0
Postcode         0
Bedroom2         0
Bathroom         0
Car              0
Landsize         0
BuildingArea     0
YearBuilt        0
CouncilArea      0
Lattitude        0
Longtitude       0
Regionname       0
Propertycount    0
dtype: int64

<div id="bir"  style="color:blue;"> 1. Veri setini yukarıdaki aşamalardan geçirdikten sonra bazı özellikler seçerek oluşturuyoruz</div>

In [53]:
features= ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea','YearBuilt', 'Lattitude', 'Longtitude']
X=data[features]

In [54]:
y=data.Price

<div id="iki"  style="color:blue;"> 2. train_test_split kullanarak modeli train ve test olarak parçalama</div>
<ul>
 <li>Modeli eğittikten sonra doğrulama(validation) yapmak için veri setini parçalara ayırıyoruz </li>
<li>random_state=0 ile her çalıştırma da veriyi aynı şekilde parçalamasını sağlıyoruz </li>
<li>Default olarak test_size=0,25 train_size=0,75 seçilidir</li>
</ul>


In [55]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0)

In [56]:
X_train.describe()

,Rooms,Bathroom,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude
count,4647.000000,4647.000000,4647.000000,4647.000000,4647.000000,4647.000000,4647.000000
mean,2.942113,1.577147,478.649451,141.531786,1964.451474,-37.808104,144.990970
std,0.972881,0.713032,963.944034,84.219417,38.401432,0.076552,0.099908
min,1.000000,1.000000,0.000000,0.000000,1196.000000,-38.164920,144.542370
25%,2.000000,1.000000,158.000000,91.000000,1940.000000,-37.855890,144.926290
50%,3.000000,1.000000,375.000000,124.000000,1970.000000,-37.801300,144.996100
75%,4.000000,2.000000,629.000000,170.000000,2000.000000,-37.758000,145.053850
max,8.000000,8.000000,37000.000000,1561.000000,2018.000000,-37.483810,145.526350


In [57]:
X_train.head()

,Rooms,Bathroom,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude
10385,3,1.0,206.0,110.0,1980.0,-37.87107,145.04991
5805,2,1.0,0.0,73.0,2000.0,-37.85900,144.97670
8488,2,1.0,2701.0,79.0,2011.0,-37.81090,144.86840
6672,3,1.0,670.0,116.0,1940.0,-37.81340,144.87450
776,6,3.0,708.0,275.0,1988.0,-37.91810,145.04400


<div id="uc"  style="color:blue;"> 3. Karar ağacı modeli oluşturma</div>
<ul>
 <li>Burada modeli eğitirken train ve test için ayırdığımız verileri kullanıyoruz</li>
<li>X_train -> Eğitim için ayrılan veri seti</li>
    <li>y_train -> Eğitim için ayrılan veri etiketi</li>
<li>Default olarak test_size=0,25 train_size=0,75 seçilidir</li>
</ul>


In [58]:
model=DecisionTreeRegressor(random_state=0)

In [59]:
model.fit(X_train,y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=0, splitter='best')

<div id="dort"  style="color:blue;"> 4. Veri seti test</div>
<ul>
 <li>Modeli eğittikten sonra test için ayırdığımız hiç eğitime sokmadığımız veri seti ile test ediyoruz</li>
</ul>

In [60]:
pred=model.predict(X_test)

<div id="bes"  style="color:blue;"> 5. Model kalitesini hata oranı ile ölçme</div>
<ul>
    <li>MAE iki süreli değişken arasındaki hatanın ölçüsüdür. Gerçek değer ile modelin tahmini arasındaki uzaklığı gösterir. </li>
    <img src="https://datavedas.com/wp-content/uploads/2018/04/image017.png"  width=300 title="Karar Ağacı Örnek Yapısı" />
    
 <li>Modelimizdeki hata oranı tespit etmek için farklı yöntemler vardır. Burada mean absolute error kullanılmıştır.</li>
   <li> Burada hata oranını tespit ederken test ettiğimiz verilerin gerçek değerlere yakınlığını ölçüyoruz</li>
</ul>

In [62]:
val=mean_absolute_error(y_test,pred)

In [63]:
print(val)

265345.3511943189
